# NLTK Chapter 9

## Building Feature Based Grammars

*The html version of this chapter in the book is available [here](https://www.nltk.org/book/ch09.html "ch09").*

### 1   Grammatical Features

We're now going to declare the features of words and phrases.  Here's an example of using dictionaries to store features and their values:

In [1]:
kim = {'CAT': 'NP', 'ORTH': 'Kim', 'REF': 'k'}
chase = {'CAT': 'V', 'ORTH': 'chased', 'REL': 'chase'}

__Feature structures__ are the pairings of features and values.

Adding more properties to the verb *chase*.  The subject plays the role of "agent" and the object the role of "patient".

In [2]:
chase['AGT'] = 'sbj'
chase['PAT'] = 'obj'

The rather convoluted code that follows processes the sentence *Kim chased Lee* and "binds" the verb's agent role to the subject (because it's to the left of the verb) and patient role to the object (because it's to the right of the verb).

In [3]:
sent = "Kim chased Lee"
tokens = sent.split()
lee = {'CAT': 'NP', 'ORTH': 'Lee', 'REF': 'l'}

def lex2fs(word):
    for fs in [kim, lee, chase]:
        if fs['ORTH'] == word:
            return fs
        
subj, verb, obj = lex2fs(tokens[0]), lex2fs(tokens[1]), lex2fs(tokens[2])
verb['AGT'] = subj['REF']
verb['PAT'] = obj['REF']

for k in ['ORTH', 'REL', 'AGT', 'PAT']:
    print("%-5s => %s" % (k, verb[k]))

ORTH  => chased
REL   => chase
AGT   => k
PAT   => l


#### 1.1   Syntactic Agreement

This simple grammar would permit *This dog runs*, but it would also permit sentences where the noun and verb don't agree, such as <i>*Theses dogs runs<i>.

In [4]:
import nltk

simple_grammar = nltk.CFG.fromstring("""
S -> NP VP
NP -> Det N 
VP -> V
Det -> 'this' | 'these'
N -> 'dog' | 'dogs'
V -> 'runs'
""")

This more complex grammar would block sentences without agreement:

In [5]:
complex_grammar = nltk.CFG.fromstring("""
S -> NP_SG VP_SG
S -> NP_PL VP_PL
NP_SG -> Det_SG N_SG
NP_PL -> Det_PL N_PL
VP_SG -> V_SG
VP_PL -> V_PL

Det_SG -> 'this'
Det_PL -> 'these'
N_SG -> 'dog'
N_PL -> 'dogs'
V_SG -> 'runs'
V_PL -> 'run'
""")

In effect, we've doubled the size of our grammar.  If we needed agreement for first, second and thrid person as well, this method would result in a grammar that would be six times the size of our original `simple` grammar.  We will look at ways of streamlining this.

#### 1.2 Using Attributes and Constraints

We could add __features__ to our notation, and use variables over values to reduce the number of required productions.

```
S[NUM = ?n] -> NP[NUM = ?n] VP[NUM = ?n]
NP[NUM = ?n] -> Det[NUM = ?n] N[NUM = ?n]
VP[NUM = ?n] -> V[NUM = ?n]
Det[NUM = sg] -> 'this'
Det[NUM = pl] -> 'these'
N[NUM = sg] -> 'dog'
N[NUM = pl] -> 'dogs'
V[NUM = sg] -> 'runs'
V[NUM = pl] -> 'run'
```

Naturally, there has to be agreement now, or the parse will fail.  

For words that agree with all numbers (i.e., singular and plural), we can leave the `NUM` value __underspecified__ instead of declaring it twice.

```
S[NUM = ?n] -> NP[NUM = ?n] VP[NUM = ?n]
NP[NUM = ?n] -> Det[NUM = ?n] N[NUM = ?n]
VP[NUM = ?n] -> V[NUM = ?n]

Det[NUM = sg] -> 'this'
Det[NUM = pl] -> 'these'
N[NUM = sg] -> 'dog'
N[NUM = pl] -> 'dogs'
V[NUM = sg] -> 'runs'
V[NUM = pl] -> 'run'
Det[NUM = ?n] -> 'the' | 'some' | 'any'
```

Most of the ideas from this chapter - and a few more - can be found in the grammar below:

In [8]:
nltk.data.show_cfg('grammars/book_grammars/feat0.fcfg')

% start S
# ###################
# Grammar Productions
# ###################
# S expansion productions
S -> NP[NUM=?n] VP[NUM=?n]
# NP expansion productions
NP[NUM=?n] -> N[NUM=?n] 
NP[NUM=?n] -> PropN[NUM=?n] 
NP[NUM=?n] -> Det[NUM=?n] N[NUM=?n]
NP[NUM=pl] -> N[NUM=pl] 
# VP expansion productions
VP[TENSE=?t, NUM=?n] -> IV[TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> TV[TENSE=?t, NUM=?n] NP
# ###################
# Lexical Productions
# ###################
Det[NUM=sg] -> 'this' | 'every'
Det[NUM=pl] -> 'these' | 'all'
Det -> 'the' | 'some' | 'several'
PropN[NUM=sg]-> 'Kim' | 'Jody'
N[NUM=sg] -> 'dog' | 'girl' | 'car' | 'child'
N[NUM=pl] -> 'dogs' | 'girls' | 'cars' | 'children' 
IV[TENSE=pres,  NUM=sg] -> 'disappears' | 'walks'
TV[TENSE=pres, NUM=sg] -> 'sees' | 'likes'
IV[TENSE=pres,  NUM=pl] -> 'disappear' | 'walk'
TV[TENSE=pres, NUM=pl] -> 'see' | 'like'
IV[TENSE=past] -> 'disappeared' | 'walked'
TV[TENSE=past] -> 'saw' | 'liked'


Using this grammar with a parser:

In [9]:
tokens = 'Kim likes children'.split()
from nltk import load_parser
cp = load_parser('grammars/book_grammars/feat0.fcfg', trace = 2)
for tree in cp.parse(tokens):
    print(tree)

|.Kim .like.chil.|
Leaf Init Rule:
|[----]    .    .| [0:1] 'Kim'
|.    [----]    .| [1:2] 'likes'
|.    .    [----]| [2:3] 'children'
Feature Bottom Up Predict Combine Rule:
|[----]    .    .| [0:1] PropN[NUM='sg'] -> 'Kim' *
Feature Bottom Up Predict Combine Rule:
|[----]    .    .| [0:1] NP[NUM='sg'] -> PropN[NUM='sg'] *
Feature Bottom Up Predict Combine Rule:
|[---->    .    .| [0:1] S[] -> NP[NUM=?n] * VP[NUM=?n] {?n: 'sg'}
Feature Bottom Up Predict Combine Rule:
|.    [----]    .| [1:2] TV[NUM='sg', TENSE='pres'] -> 'likes' *
Feature Bottom Up Predict Combine Rule:
|.    [---->    .| [1:2] VP[NUM=?n, TENSE=?t] -> TV[NUM=?n, TENSE=?t] * NP[] {?n: 'sg', ?t: 'pres'}
Feature Bottom Up Predict Combine Rule:
|.    .    [----]| [2:3] N[NUM='pl'] -> 'children' *
Feature Bottom Up Predict Combine Rule:
|.    .    [----]| [2:3] NP[NUM='pl'] -> N[NUM='pl'] *
Feature Bottom Up Predict Combine Rule:
|.    .    [---->| [2:3] S[] -> NP[NUM=?n] * VP[NUM=?n] {?n: 'pl'}
Feature Single Edge Fundame

#### 1.3   Terminology

Values like `sg` and `pl` are __atomic__ because they can't be decomposed into subparts.  We can use a __boolean__ value to distinguish __auxiliary__ verbs (e.g., modal verbs) with the boolean feature `AUX`.  For example, *can* could be represented with the production `V[TENSE=pres, AUX=+]`, though the convention is to put the `+/-` sign in front of the feature, like so: 

```
V[TENSE=pres, +AUX] -> 'can'
V[TENSE=pres, +AUX] -> 'may'

V[TENSE=pres, -AUX] -> 'walks'
V[TENSE=pres, -AUX] -> 'likes'
```

We could also group together the features in an __attribute value matrix__ (AVM):

```
[POS = N           ]
[                  ]
[AGR = [PER = 3   ]]
[      [NUM = pl  ]]
[      [GND = fem ]]
```

Order does not matter in representation, so this would be equivalent:

```
[AGR = [NUM = pl  ]]
[      [PER = 3   ]]
[      [GND = fem ]]
[                  ]
[POS = N           ]
```

We could then refactor a grammar so that agreement features are bundled together like so:

```S                    -> NP[AGR=?n] VP[AGR=?n]
NP[AGR=?n]           -> PropN[AGR=?n]
VP[TENSE=?t, AGR=?n] -> Cop[TENSE=?t, AGR=?n] Adj

Cop[TENSE=pres,  AGR=[NUM=sg, PER=3]] -> 'is'
PropN[AGR=[NUM=sg, PER=3]]            -> 'Kim'
Adj                                   -> 'happy'
```

### 2   Processing Feature Structures